In [ ]:
import sys
import os
from icecream import ic

from pathlib import Path

sys.path.insert(0, "..")
sys.path.insert(0, "../../../Utilities")

sys.path.insert(0, "../../..")

import Ballpushing_utils
import Utils
import Processing
import HoloviewsTemplates

import importlib

import holoviews as hv

hv.extension("bokeh")

# Get the list of experiments

In [ ]:
# Get the data path
Datapath = Utils.get_data_path()

# Get all folders with "TNT_Fine" in the name

Folders = [
    f for f in os.listdir(Datapath) if "TNT_Fine" in f and "Tracked" in f and os.path.isdir(Datapath / f)
]

Folders

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
# Generate Experiment objects from each folder

Experiments = [Ballpushing_utils.Experiment(Datapath / f) for f in Folders]

In [ ]:
# Check some flies nicknames

TestFly = Experiments[15].flies[3].nickname

In [ ]:
TestFly

In [ ]:
savepath = Utils.get_labserver() / "Experimental_data/MultiMazeRecorder/Datasets/240227_TNT_Fine_Experiments.pkl"


In [ ]:
Ballpushing_utils.save_object(Experiments, savepath.as_posix())

In [ ]:
# Load the experiments from the saved file
Experiments = Ballpushing_utils.load_object(savepath.as_posix())

In [ ]:
type(Experiments)

In [ ]:
type(Experiments[0])

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
data = Ballpushing_utils.Dataset(Experiments)

In [ ]:
print(data)

In [ ]:
# For each fly in the dataset, if they have 2 nicknames, just keep the first one
# for fly in data.flies:
#     if len(fly.nickname) > 1:
#         fly.nickname = fly.nickname[0]

I used the above method as a hack to get rid of a supplementary nickname in PR flies. It is fixed directly in the brain region registry now and doesn't need to be used anymore.

In [ ]:
data.generate_dataset("summary")

In [ ]:
data.jitter_boxplot(data.data, "FinalEvent", show=True, save=False)

In [ ]:
data.data

In [ ]:
# Make a subset of the data that removes None rows from "FinalEvent"

subsetfinal = data.data.dropna(subset=["FinalEvent"])

In [ ]:
data.jitter_boxplot(subsetfinal, "FinalEvent", show=False, save=True)

# New plotting method

In [ ]:
importlib.reload(HoloviewsTemplates)

In [ ]:
HoloviewsTemplates.jitter_boxplot(data.data, "FinalEvent", show=True, save=False, metadata=data.metadata, bs_controls=True)

In [ ]:
Clean = data.data.groupby("Brain region").filter(
    lambda x: x["FinalEvent"].notna().any()
)

# Clean the data by removing NaN values for this metric
Clean = Clean.dropna(subset=["FinalEvent"])

Clean["FinalEvent"] = pd.to_numeric(Clean["FinalEvent"], errors="coerce")

hv.BoxWhisker(
            data=Clean,
            vdims="FinalEvent",
            kdims=["label", "Brain region"],
        ).groupby("Brain region")

In [ ]:
Clean = data.data.groupby("Brain region").filter(
    lambda x: x["FinalEvent"].notna().any()
)

# Clean the data by removing NaN values for this metric
Clean = Clean.dropna(subset=["FinalEvent"])


In [ ]:
print(Clean["FinalEvent"].dtype)

In [ ]:
import pandas as pd

Clean["FinalEvent"] = pd.to_numeric(Clean["FinalEvent"], errors="coerce")
boxplot = hv.BoxWhisker(
    data=Clean,
    vdims="FinalEvent",
    kdims=["label", "Brain region"],
)

boxplot

In [ ]:
hv.ipython.show_traceback()

In [ ]:
print(data.data.groupby("Brain region").filter(lambda x: x["FinalEvent"].notna().any()))

# Troubleshooting the data error

In my dataset I get this: 
AbbreviatedException: ValueError: List parameter 'PipelineMeta.vdims' length must be between 1 and 1 (inclusive), not 0.


only with some metrics. In order to troubleshoot this, let's generate dataset objects with incrementally more experiments to see where the error comes from.

In [ ]:
# Using the list of experiments, in a loop generate a dataset including one more experiment each time and try to plot the jitter boxplot. If that works, proceed to the next step. If that doesn't work, print the name of the last experiment that was added to the dataset.

for exp in Experiments:
    try:
        data = Ballpushing_utils.Dataset(Experiments[: Experiments.index(exp) + 1])
        subsetfinal = data.data.dropna(subset=["FinalEvent"])
        data.jitter_boxplot(subsetfinal, "FinalEvent", show=False, save=True)
    
    except:
        print(f"Error with {exp.directory}")
    

Seems its a problem with all of the experiments, however if I import them differently, with their paths, it works. Let's double check that

In [ ]:
testdata = Ballpushing_utils.Dataset(
    [
        Ballpushing_utils.Experiment(
            Path(
                "/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos/231121_TNT_Fine_1_Videos_Tracked/"
            )
        ),
        Ballpushing_utils.Experiment(
            Path(
                "/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos/231121_TNT_Fine_2_Videos_Tracked/"
            )
        ),
    ]
)

testdata.generate_dataset("summary")

testdata.jitter_boxplot(testdata.data, "FinalEvent", save=True)

Maybe the issue is specific to the control thing? I see it's not in my working example. Let's extract the function and remove the control thing and see if it works.

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
data.jitter_boxplot(data.data, "FinalEvent", save=False, show=True)

# PCA on the data

Here I'll try to do PCA on the data to see if I can get something interesting by reducing the dimensionality of the data, including all the summary metrics.

In [ ]:
# Subset the data to only include the label and metrics of interest
subset = data.data[
    [
        "NumberEvents",
        "FinalEvent",
        "FinalTime",
        "SignificantEvents",
        "SignificantFirst",
        "SignificantFirstTime",
        "CumulatedBreaks",
        "Pushes",
        "Pulls",
        "Genotype",
        "Brain region",
    ]
]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import hvplot.pandas

# Separate the "TNTxZ2018" data from the rest of the data
TNTxZ2018_data = PCA_components[PCA_components["Genotype"] == "TNTxZ2018"]
other_data = PCA_components[PCA_components["Genotype"] != "TNTxZ2018"]

# Generate one plot per Brain region
for brain_region in PCA_components["Brain region"].unique():
    df_brain_region = other_data[other_data["Brain region"] == brain_region]

    # Create separate scatter plots for the "TNTxZ2018" genotype and the other genotypes
    plot1 = df_brain_region.hvplot.scatter(
        x="PC1", y="PC2", by="Genotype", hover_cols=["Genotype"], cmap="nipy_spectral"
    )
    plot2 = TNTxZ2018_data.hvplot.scatter(
        x="PC1",
        y="PC2",
        by="Genotype",
        hover_cols=["Genotype"],
        color="black",
        marker="x",
        size=100,
    )

    # Combine the plots
    final_plot = plot1 * plot2

    # Display the plot
    hvplot.show(final_plot)

In [ ]:
subset

In [ ]:
import hvplot.pandas

# Create a scatter plot for each brain region and genotype
plot = PCA_components.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Genotype",
    groupby="Brain region",
    hover_cols=["Genotype"],
    cmap="nipy_spectral",
    dynamic=False,
)

# Adjust the appearance of the "TNTxZ2018" points
plot.opts(hv.opts.Scatter("TNTxZ2018", size=100, marker="x", color="black"))

# Display the plot
hvplot.show(plot)

In [ ]:
import hvplot.pandas
import holoviews as hv

# Separate the "TNTxZ2018" data from the rest of the data
TNTxZ2018_data = PCA_components[PCA_components["Genotype"] == "TNTxZ2018"]
other_data = PCA_components[PCA_components["Genotype"] != "TNTxZ2018"]

# Initialize an empty Layout
plots = hv.Layout()

# Generate one plot per Brain region
for brain_region in PCA_components["Brain region"].unique():
    df_brain_region = other_data[other_data["Brain region"] == brain_region]

    # Create separate scatter plots for the "TNTxZ2018" genotype and the other genotypes
    plot1 = df_brain_region.hvplot.scatter(
        x="PC1", y="PC2", by="Genotype", hover_cols=["Genotype"], cmap="nipy_spectral"
    )
    plot2 = TNTxZ2018_data.hvplot.scatter(
        x="PC1",
        y="PC2",
        by="Genotype",
        hover_cols=["Genotype"],
        color="black",
        marker="x",
        size=100,
    )

    # Combine the plots
    final_plot = plot1 * plot2

    # Add the plot to the Layout
    plots += final_plot.relabel(f"PCA - Brain Region: {brain_region}")

# Save the Layout
hvplot.save(plots.cols(1), "/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Plots/240222_PCA_plots.html")
# Display the Layout
hvplot.show(plots.cols(1))